# Initialization

In [1]:
import os
import itertools
import sys
from math import factorial as fac
sys.path.append("D:/TU_Delft/Msc_Building_Technology/Semester_3/Graduation/Aditya_Graduation_Project_BT/06_Libraries")
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import networkx as nx
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)
import networkx as nx
from itertools import combinations
from itertools import permutations
import pickle
import pandas as pd

In [2]:
lattice_path = os.path.relpath('Zoning_envelop.csv')
Zoning_matrix_sequential = tg.lattice_from_csv(lattice_path)

In [3]:
lattice_path_availability_matrix = os.path.relpath('voxelized_envelope_6m_voxel_size.csv')
avail_lattice_base = tg.lattice_from_csv(lattice_path_availability_matrix)
avail_lattice = avail_lattice_base*1
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)

# Stencils

In [4]:
stencil_von_neumann = tg.create_stencil("von_neumann", 1, 1)
stencil_von_neumann.set_index([0,0,0], 0)

In [5]:
stencil_cuboid = tg.create_stencil("moore", 1, 1)
stencil_cuboid.set_index([0,0,0], 0)

In [6]:
# creating neighborhood definition 
stencil_squareness = tg.create_stencil("moore", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness[0,:,:] = 0 
stencil_squareness[2,:,:] = 0
stencil_squareness.set_index([0,0,0], 0)
stencil_squareness_t = np.transpose(stencil_squareness) 
#print(stencil_squareness_t)

# Initial Vizualization

In [7]:
p = pv.Plotter(notebook=True)

base_lattice = Zoning_matrix_sequential
 
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

init_avail_lattice.fast_vis(p)
# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

# Add the data values to the cell data
grid.cell_arrays["Agents"] = Zoning_matrix_sequential.flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([101,210])

# adding the voxels
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)

p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(168.72367820989257, 336.72367820989257, 303.72367820989257),
 (-105.0, 63.0, 30.0),
 (0.0, 0.0, 1.0)]

# Cleanup 

In [8]:
def cleanup_algorithm (Zoning_matrix,stencil_1,stencil_2,a_id):
    #First Cleanup
    Zoning_matrix_flat = Zoning_matrix.flatten()
    all_indices_for_agent = np.argwhere(Zoning_matrix_flat ==a_id).flatten()
    all_vox_neighs_inds = Zoning_matrix.find_neighbours(stencil_1)[np.argwhere(Zoning_matrix_flat ==a_id)]
    retrieved_neighs= Zoning_matrix_flat[all_vox_neighs_inds]
    #Second Cleanup
    for_improvement = np.copy(Zoning_matrix_flat)
    for_improvement[np.argwhere(Zoning_matrix_flat==0)] =a_id
    all_vox_neighs_inds_improv = Zoning_matrix.find_neighbours(stencil_2)[np.argwhere(Zoning_matrix_flat ==a_id)]
    retrieved_neighs_improv = for_improvement[all_vox_neighs_inds_improv]
    
    indexing =[]
    for i,  item in enumerate(retrieved_neighs_improv):
        flattened_list = item.flatten()
        #print(flattened_list)
        if np.all(flattened_list[[0,1,2,7,3]] == a_id) or np.all(flattened_list[[3,4,5,6,7]] == a_id) or np.all(flattened_list[[1,2,3,4,5]] == a_id) or np.all(flattened_list[[5,6,7,0,1]] == a_id):
            a = i
        else:
            indexing.append(i)
            #print("truthy")
    lattice_indexes= all_indices_for_agent[indexing]
    return retrieved_neighs,all_indices_for_agent, lattice_indexes

In [9]:
PH_Unoccupy_indexes = cleanup_algorithm(Zoning_matrix_sequential,stencil_cuboid,stencil_squareness_t,200)

# Unoccupy process

In [10]:
def unoccupy_based_on_number(lattice,removal_indices,a_id,number):
    list_of_neighbour_repetitions = (np.sum(np.count_nonzero(removal_indices[0] == a_id,axis =1),axis=1))
    removal_indices= removal_indices[1][np.argwhere(list_of_neighbour_repetitions <= number)].flatten()
    lattice[np.unravel_index(removal_indices,avail_lattice.shape)] = 0

In [11]:
unoccupy_based_on_number (Zoning_matrix_sequential,PH_Unoccupy_indexes,200,6)

In [12]:
#def unoccupy_based_on_stencil(lattice,removal_indices):
Zoning_matrix_sequential[np.unravel_index(PH_Unoccupy_indexes[2],Zoning_matrix_sequential.shape)] = 0

In [13]:
#unoccupy_based_on_stencil( Zoning_matrix_sequential,PH_Unoccupy_indexes[2])

In [14]:
PH_Unoccupy_indexes[2]

array([ 722,  723,  733,  734,  897,  900,  908, 1072, 1083, 1247, 1258,
       1422, 1433, 1597, 1608, 1729, 1772, 1783, 1895, 1906, 1912, 2072,
       2073, 2074, 2075, 2083, 2084, 2085, 2086, 2087, 5949, 5971, 6124,
       6146], dtype=int64)

# Final Viz after cleanup 

In [15]:
p = pv.Plotter(notebook=True)

base_lattice = Zoning_matrix_sequential
 
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

init_avail_lattice.fast_vis(p)
# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

# Add the data values to the cell data
grid.cell_arrays["Agents"] = Zoning_matrix_sequential.flatten(order="F").astype(int)  # Flatten the array!
# filtering the voxels
threshed = grid.threshold([101,210])

# adding the voxels
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)

p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(168.72367820989257, 336.72367820989257, 303.72367820989257),
 (-105.0, 63.0, 30.0),
 (0.0, 0.0, 1.0)]

In [16]:
n = 8
n1 = 5
perms =[]
for x in itertools.combinations( range(n), n1 ) :
    perms.append([ 1 if i in x else 0 for i in range(n)] )

In [17]:
perms

[[1, 1, 1, 1, 1, 0, 0, 0],
 [1, 1, 1, 1, 0, 1, 0, 0],
 [1, 1, 1, 1, 0, 0, 1, 0],
 [1, 1, 1, 1, 0, 0, 0, 1],
 [1, 1, 1, 0, 1, 1, 0, 0],
 [1, 1, 1, 0, 1, 0, 1, 0],
 [1, 1, 1, 0, 1, 0, 0, 1],
 [1, 1, 1, 0, 0, 1, 1, 0],
 [1, 1, 1, 0, 0, 1, 0, 1],
 [1, 1, 1, 0, 0, 0, 1, 1],
 [1, 1, 0, 1, 1, 1, 0, 0],
 [1, 1, 0, 1, 1, 0, 1, 0],
 [1, 1, 0, 1, 1, 0, 0, 1],
 [1, 1, 0, 1, 0, 1, 1, 0],
 [1, 1, 0, 1, 0, 1, 0, 1],
 [1, 1, 0, 1, 0, 0, 1, 1],
 [1, 1, 0, 0, 1, 1, 1, 0],
 [1, 1, 0, 0, 1, 1, 0, 1],
 [1, 1, 0, 0, 1, 0, 1, 1],
 [1, 1, 0, 0, 0, 1, 1, 1],
 [1, 0, 1, 1, 1, 1, 0, 0],
 [1, 0, 1, 1, 1, 0, 1, 0],
 [1, 0, 1, 1, 1, 0, 0, 1],
 [1, 0, 1, 1, 0, 1, 1, 0],
 [1, 0, 1, 1, 0, 1, 0, 1],
 [1, 0, 1, 1, 0, 0, 1, 1],
 [1, 0, 1, 0, 1, 1, 1, 0],
 [1, 0, 1, 0, 1, 1, 0, 1],
 [1, 0, 1, 0, 1, 0, 1, 1],
 [1, 0, 1, 0, 0, 1, 1, 1],
 [1, 0, 0, 1, 1, 1, 1, 0],
 [1, 0, 0, 1, 1, 1, 0, 1],
 [1, 0, 0, 1, 1, 0, 1, 1],
 [1, 0, 0, 1, 0, 1, 1, 1],
 [1, 0, 0, 0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 1, 0, 0],
 [0, 1, 1, 1, 1, 0, 1, 0],
 

In [18]:
perms[0]

[1, 1, 1, 1, 1, 0, 0, 0]